In [1]:
#@markdown # 🔧 Run Setup

!pip install yt-dlp mega.py
import requests
from IPython.display import clear_output
import os
from google.colab import drive
from mega import Mega
clear_output()

In [ ]:
#@markdown  # 📥 Mount Google drive
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!

drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [ ]:
#@markdown  # 📥 YouTube Playlist Downloader 📥

#@markdown This notebook uses [yt-dlp](https://github.com/yt-dlp/yt-dlp) to download a YouTube playlist.


#@markdown ## 📝 Enter Playlist URL

from functools import wraps
import errno
import os
import signal
import subprocess
import shlex
import requests
from datetime import datetime

def notify(text='Download Finished'):

  token = 'BOT_TOKEN'
  data = {'chat_id': 'CHAT_ID', 'text': text}
  url = f'https://api.telegram.org/bot{token}/sendMessage'
  response = requests.get(url, params=data)
  print(response.json())



if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh



def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

def runShT(args):
    return runSh(args, output=True)


PLAYLIST = "https://www.youtube.com/playlist?list=PL3JVwFmb_BnTamTxXbmlwpspYdpmaHdbz"  # @param {type:"string"}

OUTPUT_FOLDER = "Google drive" # @param {type:"string"}
MOVE_TO_FOLDER = "/content/drive/MyDrive/youtube" # @param {type:"string"}

USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}



UPLOAD_MEGA = False # @param {type:"boolean"}
SUBTITLE = True # @param {type:"boolean"}
ZIP = True # @param {type:"boolean"}
NOTIFY_TEL = True # @param {type:"boolean"}

#Clean up previous unsuccessfull or unfinished download if existed

try:
  runShT(f"rm -rf {OUTPUT_FOLDER}")
except Exception as e:
  print(e.args)
  pass

try:


  mega = Mega()
  m = mega.login(USERNAME , PASSWORD)

  start = datetime.now()
  print(f"------------Process started {start} ------------")

  if NOTIFY_TEL:
    notify(f"------------Process started {start} ------------")

  cmd = "yt-dlp -f 'bv*+ba/b' "


  if OUTPUT_FOLDER:
    OUTPUT_FOLDER = OUTPUT_FOLDER.replace(" ","-")
    os.makedirs(OUTPUT_FOLDER,exist_ok=True)
    cmd = cmd + "-o " + f"{OUTPUT_FOLDER}/%(title)s.%(ext)s"

  if SUBTITLE:
    subs = "--write-auto-sub  --sub-lang en,fa  --embed-sub "
    cmd = cmd + subs

  cmd = cmd + PLAYLIST
  print(f"executing {cmd}")
  print("Downloading playlist now...")


  runShT(cmd)


  print("✅ Done!")
  clear_output(wait=5)

  if ZIP:
    print("Zipping files now...")
    runShT(f"zip -rv '{OUTPUT_FOLDER}.zip' '{OUTPUT_FOLDER}' ")
    OUTPUT_FOLDER = f'{OUTPUT_FOLDER}.zip'
    clear_output(wait=5)

  if MOVE_TO_FOLDER:

    runShT(f"mv {OUTPUT_FOLDER}  {MOVE_TO_FOLDER}")


  if UPLOAD_MEGA:
    print("Zipping finished uploading to mega.nz")
    print((m.upload(filename=OUTPUT_FOLDER)))
    file = m.find(OUTPUT_FOLDER)

    if file:
        # get public link
        link = m.get_link(file)
        print(link)


  end = start - datetime.now()

  if NOTIFY_TEL:
    notify(f"------------Process ✅ Done in  {end} seconds ------------")

except Exception as e:
  print("❌",e.args)